In [1]:
"""
This file generates queries given input tables, text and annotations data. We evaluate the results of the queries
based on the input tables
"""

'\nThis file generates queries given input tables, text and annotations data. We evaluate the results of the queries\nbased on the input tables\n'

In [82]:
import pandas as pd
import pandasql as ps
import os
import math

In [83]:
TABLES = "../data/demo_tables/"

In [84]:
table1_path = os.path.join(TABLES, "demo_table1.csv")
table1_df = pd.read_csv(table1_path)
table1_df.rename(columns={"Unnamed: 0": "row_name"}, inplace=True)
table1_df.name = "table1_df"

In [85]:
table1_df

,row_name,2016,2017e,2030,2035,2040
0,North America,4425,5680,1120,5653,1550
1,Central & South America,2330,3920,2131,1375,3045
2,Europe,2708,2376,2053,887,3670
3,Africa,2257,1305,2217,5788,2922
4,Middle East,4151,5830,2033,4962,1402
5,Eurasia,4664,4901,1262,2096,3242
6,Asia Pacific,5304,3025,5624,2718,13690
7,World,1328,25679,3570,4582,37114


In [6]:
table2_path = os.path.join(TABLES, "demo_table2.csv")
table2_df = pd.read_csv(table2_path)
table2_df.rename(columns={"Unnamed: 0": "row_name"}, inplace=True)
table2_df.name = "table2_df"

In [7]:
table2_df

,row_name,2016,2017e,2025,2030,2035,2040
0,CO2 emissions (Mt),3851,4956,5776,5927,1758,3187
1,Intensity (g CO2/kWh),825,484,1330,1995,3676,69


In [8]:
table3_path = os.path.join(TABLES, "demo_table3.csv")
table3_df = pd.read_csv(table3_path)
table3_df.rename(columns={"Unnamed: 0": "row_name"}, inplace=True)
table3_df.name = "table3_df"

In [9]:
table3_df

,row_name,2000,2016,2017e,2025,2030,2035,2040
0,Total CO2,28219,33228,25202,32830,29960,29705,35881
1,Coal,9683,13494,5875,12224,12373,9160,15327
2,Oil,6923,5953,6560,8427,8257,6427,11315
3,Gas,11613,13781,12767,12179,9330,14118,11950


In [10]:
type(table3_df.iloc[0]["row_name"])

str

In [114]:
demo_query = "SELECT t1.'2035' from {} as t1 WHERE t1.'row_name' == 'North America'".format(table1_df.name)

In [115]:
a = ps.sqldf(demo_query, locals())

In [116]:
a

,2035
0,2625.1


In [117]:
type(a.values.tolist()[0][0]), a.shape 

(float, (1, 1))

In [13]:
a.iloc[0]["2016"]

1840

In [22]:
list(table1_df.columns)

['row_name', '2016', '2017e', '2030', '2035', '2040']

In [15]:
def execute_query(query):
    """
    executes the query and returns a Dataframe
    """
    try:
        df = ps.sqldf(query, locals())
        return df
    except:
        return 

In [88]:
def existence_query_generation(df):
    queries_list = []
    columns = list(df.columns)
    rows = list(df.row_name)
    for column in columns:
        for row in rows:
            query = "SELECT t.'{}' from {} as t WHERE t.'row_name' == '{}'".format(column, df.name, row)
            queries_list.append(query)
            
    print("Generated {} existence queries".format(len(queries_list)))
    return queries_list
            

In [204]:
def get_column_tuples(columns):
    """
    given a columns list ["a", "b", "c", "d"] return a list
    [('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')]
    """
    tuple_list = []
    for idx, col in enumerate(columns):
        t = tuple(columns[idx:])
        first_item = t[0]
        for a in t[1:]:
            tuple_list.append((first_item, a))
    return tuple_list

In [207]:
def percentage_increase_query_generation(df):
    queries_list = []
    columns = list(df.columns)
    rows = list(df.row_name)
    column_tuples = get_column_tuples(columns)
    for column_tuple in column_tuples:
        for row in rows:
            query = "SELECT (t.'{}' - t.'{}')/t.'{}' from {} as t WHERE t.'row_name' == '{}'".format(column_tuple[1], column_tuple[0], column_tuple[0], df.name, row)
            queries_list.append(query)
            
    print("Generated {} percentage increase queries".format(len(queries_list)))
    return queries_list

In [208]:
percentage_increase_query_generation(table2_df)

Generated 42 percentage increase queries


["SELECT (t.'2016' - t.'row_name')/t.'row_name' from table2_df as t WHERE t.'row_name' == 'CO2 emissions (Mt)'",
 "SELECT (t.'2016' - t.'row_name')/t.'row_name' from table2_df as t WHERE t.'row_name' == 'Intensity (g CO2/kWh)'",
 "SELECT (t.'2017e' - t.'row_name')/t.'row_name' from table2_df as t WHERE t.'row_name' == 'CO2 emissions (Mt)'",
 "SELECT (t.'2017e' - t.'row_name')/t.'row_name' from table2_df as t WHERE t.'row_name' == 'Intensity (g CO2/kWh)'",
 "SELECT (t.'2025' - t.'row_name')/t.'row_name' from table2_df as t WHERE t.'row_name' == 'CO2 emissions (Mt)'",
 "SELECT (t.'2025' - t.'row_name')/t.'row_name' from table2_df as t WHERE t.'row_name' == 'Intensity (g CO2/kWh)'",
 "SELECT (t.'2030  ' - t.'row_name')/t.'row_name' from table2_df as t WHERE t.'row_name' == 'CO2 emissions (Mt)'",
 "SELECT (t.'2030  ' - t.'row_name')/t.'row_name' from table2_df as t WHERE t.'row_name' == 'Intensity (g CO2/kWh)'",
 "SELECT (t.'2035  ' - t.'row_name')/t.'row_name' from table2_df as t WHERE t.

In [90]:
def query_generation(claim_text, claim_values, df_list):
    """
    generate queries based on the input claims and the dataframes
    """
    existence_queries = []
    for df in df_list:
        existence_queries += existence_query_generation(df)
    return existence_queries

In [41]:
def int_round(x, n=10):
    res = math.ceil(x/n)*n
    if (x%n < n/2)and (x%n>0):
        res-=n
    return res

In [162]:
def get_num_digits(num):
    l = str(num).split(".")
    if len(l) == 1:
        return len(l[0])
    else:
        return len(l[0]) + len(l[1]) - 1 # floats with .0

In [163]:
def get_int_round_list(x, num_digits):
    if num_digits == 1:
        return [x]
    elif num_digits == 2:
        return [x, int_round(x, 10)]
    elif num_digits == 3:
        return [x, int_round(x, 10), int_round(x, 100)] 
    elif num_digits == 4:
        return [x, int_round(x, 10), int_round(x, 100), int_round(x, 1000)]
    elif num_digits == 5:
        return [x, int_round(x, 10), int_round(x, 100), int_round(x, 1000)]

In [192]:
get_int_round_list(112.3, 3)

[112.3, 110, 100]

In [165]:
int_round(37114.5, 1000)

37000

In [166]:
round(111, 100)

111

In [167]:
num = str(111)
num.split('.')

['111']

In [168]:
def approximately_equal(claim_value, query_value):
    """
    returns True if the claim value and the query value are approximately equal
    """
    if isinstance(query_value, str):
        return False
    else:
        num_digits_q = get_num_digits(query_value)
        num_digits_c = get_num_digits(claim_value)
        
        if claim_value in get_int_round_list(query_value, num_digits_q):
            return True
        else:
            return False

In [169]:
def estimate_query_result(claim_value, query_result):
    """
    compares the query result (Dataframe) and the claim value.
    Assume that the dataframe has only one element.
    Return True/False
    """
    if len(query_result) == 0:
        return False
    if len((query_result.columns)) != 1:
        return False
    q_col = list(query_result.columns)[0]
    query_value = query_result.iloc[0][q_col]
    try:
        query_value = float(query_value)
    except ValueError:
        query_value = query_value
        
    print(query_value, type(query_value))
    return approximately_equal(claim_value, query_value)

In [170]:
def query_execution(claim_text, claim_value, df_list):
    """
    Executes the queries generated and compares the result. If the result is close
    to the claim, then we return this query!
    """
    all_queries = []
    existence_queries = query_generation(claim_text, claim_value, df_list)
    all_queries += existence_queries
    
    # queries to be returned as possible candidates
    candidate_queries = []
    
    for query in all_queries:
        query_result = ps.sqldf(query)
        if query_result is None:
            print("lala")
            continue
        est = estimate_query_result(claim_value, query_result)
        if est:
            candidate_queries.append(query)
    
    return candidate_queries

In [187]:
claim_text = "lala"
claim_value = 70

In [188]:
query_execution(claim_text, claim_value, [table1_df, table2_df, table3_df])

Generated 48 queries
Generated 14 queries
Generated 32 queries
North America <class 'str'>
Central & South America <class 'str'>
Europe <class 'str'>
Africa <class 'str'>
Middle East <class 'str'>
Eurasia <class 'str'>
Asia Pacific <class 'str'>
World <class 'str'>
1840.0 <class 'float'>
1055.0 <class 'float'>
2822.0 <class 'float'>
3419.0 <class 'float'>
2918.0 <class 'float'>
5443.0 <class 'float'>
4348.0 <class 'float'>
895.0 <class 'float'>
1225.0 <class 'float'>
3478.0 <class 'float'>
5350.0 <class 'float'>
1474.0 <class 'float'>
5331.0 <class 'float'>
3825.0 <class 'float'>
3025.0 <class 'float'>
25 679 <class 'str'>
3345.0 <class 'float'>
2059.0 <class 'float'>
4937.0 <class 'float'>
2483.0 <class 'float'>
5508.0 <class 'float'>
2325.0 <class 'float'>
1544.0 <class 'float'>
2240.0 <class 'float'>
2625.0 <class 'float'>
2766.0 <class 'float'>
3817.0 <class 'float'>
4127.0 <class 'float'>
2700.0 <class 'float'>
5472.0 <class 'float'>
2775.0 <class 'float'>
5202.0 <class 'float'>
1

["SELECT t.'2040  ' from table2_df as t WHERE t.'row_name' == 'Intensity (g CO2/kWh)'"]

In [220]:
ps.sqldf("SELECT CAST((t.'2016' - t.'2017e')/10000 AS DECIMAL(53,8)) from table2_df as t WHERE t.'row_name' == 'CO2 emissions (Mt)'", locals())

,"CAST((t.'2016' - t.'2017e')/10000 AS DECIMAL(53,8))"
0,0


In [148]:
int("hello")

ValueError: invalid literal for int() with base 10: 'hello'

In [197]:
l = ["a", "b", "c", "d"]

In [202]:
t = []
for idx, itm in enumerate(l):
    q = tuple(l[idx:])
    first_item = q[0]
    for a in q[1:]:
        t.append((first_item, a))
    

In [203]:
t

[('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')]

In [89]:
int_round(14446, 1000)

14000

In [59]:
import numpy as np

In [77]:
num_digits = 7

In [78]:
[10*(10**i) for i in np.arange(num_digits-2)]

[10, 100, 1000, 10000, 100000]

In [92]:
x = 0.56

In [109]:
name = "table1"

In [110]:
exec(name + " = table1_df")

In [112]:
table1

,row_name,2016,2017e,2030,2035,2040
0,North America,1840,1225,3345,2625.1,1088
1,Central & South America,1055,3478,2059,2766.0,5706
2,Europe,2822,5350,4937,3817.0,4624
3,Africa,3419,1474,2483,4127.0,4155
4,Middle East,2918,5331,5508,2700.0,2540
5,Eurasia,5443,3825,2325,5472.0,4088
6,Asia Pacific,4348,3025,1544,2775.0,13690
7,World,895,25679,2240,5202.0,37114


In [189]:
perc_q = "SELECT cast ((abs(100.0*(t.'2017e' - t.'2016')/t.'2016')) as numeric(2,2)) from table1_df as t "

In [190]:
r = ps.sqldf(perc_q)

In [191]:
r

,"cast ((abs(100.0*(t.'2017e' - t.'2016')/t.'2016')) as numeric(2,2))"
0,33.423913
1,229.668246
2,89.581857
3,56.887979
4,82.693626
5,29.726254
6,30.427783
7,2769.162011


In [192]:
r.values.tolist()

[[33.42391304347826],
 [229.66824644549763],
 [89.58185683912119],
 [56.88797894121088],
 [82.69362577107609],
 [29.726253904097007],
 [30.42778288868445],
 [2769.1620111731845]]

In [115]:
q = "SELECT table1_df.'2040'/table1_df.'2017e' - table1_df.'2017e'/table1_df.'2017e' FROM table1_df"

In [116]:
ps.sqldf(q)

,table1_df.'2040'/table1_df.'2017e' - table1_df.'2017e'/table1_df.'2017e'
0,-0.727113
1,-0.223214
2,0.544613
3,1.239080
4,-0.759520
5,-0.338502
6,3.525620
7,0.445306


In [194]:
"-".join(["laa", "2"])

'laa-2'

In [40]:
["t" + "." + "'" + col + "'" for col in list(table1_df.columns)]

["t.'row_name'", "t.'2016'", "t.'2017e'", "t.'2030'", "t.'2035'", "t.'2040'"]

In [51]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [59]:
s = []
n = []

for c in table1_df.columns:
    print(c)
    if is_string_dtype(table1_df[c]):
        s.append(c)
    if is_numeric_dtype(table1_df[c]):
        n.append(c)

row_name
2016
2017e
2030
2035
2040


In [61]:
n, s

(['2016', '2017e', '2030', '2035', '2040'], ['row_name'])

In [97]:
table1_df = table1_df.astype(float, errors="ignore")

In [98]:
table1_df

,row_name,2016,2017e,2030,2035,2040
0,North America,4425.0,5680.0,1120.0,5653.0,1550.0
1,Central & South America,2330.0,3920.0,2131.0,1375.0,3045.0
2,Europe,2708.0,2376.0,2053.0,887.0,3670.0
3,Africa,2257.0,1305.0,2217.0,5788.0,2922.0
4,Middle East,4151.0,5830.0,2033.0,4962.0,1402.0
5,Eurasia,4664.0,4901.0,1262.0,2096.0,3242.0
6,Asia Pacific,5304.0,3025.0,5624.0,2718.0,13690.0
7,World,1328.0,25679.0,3570.0,4582.0,37114.0


In [102]:
round(1.3222, 3)

1.322